In [2]:
import pandas as pd

In [1]:
from minio import Minio
from minio.error import S3Error

In [2]:
minio_client = Minio(
                        endpoint='localhost:9000',
                        access_key='User',
                        secret_key='p4ssw0rd',
                        secure=False)

# Nombre del bucket en MinIO
bucket_name = 'bucketbiblos'

In [3]:
try:
    objetos = minio_client.list_objects(bucket_name, recursive=True)
    # Filtrar los objetos que tienen extensión .json
    archivos_json = [obj.object_name for obj in objetos if obj.object_name.endswith('.json')]
    # Imprimir la lista de archivos JSON
    print("Archivos JSON en el bucket:")
    for archivo in archivos_json:
        print(archivo)
except S3Error as err:
    print(err)

Archivos JSON en el bucket:
url1.json
url2.json
url3.json


In [4]:
import json

datos_json ={}
for archivo in archivos_json:
    try:
        print(archivo)
        data = minio_client.get_object(bucket_name, archivo)
    # Leer los datos del archivo
        json_data = data.read().decode('utf-8')
    # Trabajar con los datos (por ejemplo, imprimirlos)
        print(json.loads( json_data))
        datos_json[archivo] =json.loads( json_data)
    except S3Error as err:
        print(err)

url1.json
{'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'EPSG:3857'}}, 'features': [{'type': 'Feature', 'id': 13, 'geometry': {'type': 'Point', 'coordinates': [-8244653.3674, 530074.8121000007]}, 'properties': {'OBJECTID': 13, 'POINT_X': -74.06298132, 'POINT_Y': 4.75627098, 'LECCODIGO': 15, 'LECNOMBRE': 'Centro Cultural y Biblioteca Pública Julio Mario Santo Domingo', 'LECDIRECCI': 'Avenida calle 170 No. 67-51\xa0', 'LECTELEFON': '57 1 - 5803080', 'LECEMAIL': 'direccion_jm@biblored.gov.co', 'LECPAGWEB': 'http://www.biblored.gov.co/index.php/bibliotecas', 'LECCODLOC': '11', 'LECNOMLOC': 'Suba', 'LECCODUPZ': 'UPZ18', 'LECNOMUPZ': 'Britalia', 'LECCODSEC': '009134', 'LECNOMSEC': 'San José V Sector', 'LECESTADO': 'Activo', 'LECNODO': 'Nodo Julio Mario Santo Domingo', 'LECCLASIF': 'BibloRed', 'LECANIO': '2016', 'LECCONTACTO': 'N.A'}}, {'type': 'Feature', 'id': 14, 'geometry': {'type': 'Point', 'coordinates': [-8247488.8803, 518995.93659999967]}, 'properties': {

In [5]:
datos_json['url1.json'].keys()

dict_keys(['type', 'crs', 'features'])

In [6]:

for biblo in datos_json['url1.json']['features']:
    biblo=biblo['properties']
    print(biblo['OBJECTID']," ", biblo['LECNOMBRE'])

13   Centro Cultural y Biblioteca Pública Julio Mario Santo Domingo
14   Biblioteca Pública Virgilio Barco
15   Biblioteca Pública Arborizadora Alta
16   Biblioteca Pública El Tunal Gabriel García Márquez
17   Biblioteca Pública Usaquén Servita
18   Biblioteca Pública Venecia Pablo de Tarso
19   Biblioteca Pública La Giralda
20   Biblioteca Público Escolar La Marichuela
21   Biblioteca Publico Escolar de Sumapaz
22   Biblioteca Público Escolar Pasquilla
23   Biblioteca Pública El Parque
24   Biblioteca Instituto Geográfico Agustín Codazzi
25   Biblioteca Luis Carlos Galán Sarmiento - Biblioteca del Congreso
26   Biblioteca Luis Ángel Arango
27   Biblioteca Nacional de Colombia
28   Biblioteca especializada en Historia Política - FUGA
29   Biblioteca Cárcel Distrital
30   Biblioteca Universitaria de los Andes
31   Biblioteca Universitaria Jorge Tadeo Lozano
32   Biblioteca Universitaria de la Nacional
1   Biblioteca Pública del Deporte
2   Biblioteca Pública Carlos E. Restrepo
3   Bibli

In [1]:
import psycopg2

# Parámetros de conexión
dbname = 'postgres'
user = 'postgres'
password = '123postgis'
host = 'localhost'  # Cambia esto si tu base de datos está en otro host
port = '5432'  # Puerto pre determinado de PostgreSQL

# Crear una conexión
try:
    conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port)
    print("connection exitosa")
except Exception as e:
    print("Error", e)




connection exitosa


In [2]:


# Crear un cursor para ejecutar consultas
cur = conn.cursor()

# Ejecutar una consulta
cur.execute('SELECT * FROM "user"')

# Obtener los resultados
rows = cur.fetchall()
for row in rows:
    print(row)

# Cerrar el cursor y la conexión
cur.close()
conn.close()


(1, 'Usuario 1')
(2, 'Usuario 2')
(3, 'Usuario 3')
(4, 'Usuario 4')
(5, 'Usuario 5')


In [5]:
from src.app_biblio.services.DatabaseConnection import DatabaseConnection

In [11]:
# Crear una instancia de DatabaseConnection
conn = DatabaseConnection()

# Conectar a la base de datos
conn.connect()
result = conn.execute('SELECT * FROM "user"')
conn.disconnect()
result



[(1, 'Usuario 1'),
 (2, 'Usuario 2'),
 (3, 'Usuario 3'),
 (4, 'Usuario 4'),
 (5, 'Usuario 5')]

In [3]:
 import numpy as np
 np.random.randn(1000, 2) / [50, 50] + [37.76, -122.4]

array([[-0.01040196, -0.01236816],
       [ 0.01835013,  0.01841979],
       [-0.00386102,  0.02613619],
       ...,
       [-0.01330529,  0.02317854],
       [-0.01478457,  0.01193827],
       [-0.00304211,  0.04367834]])

In [10]:
def get_bibliotecas_by_id_libro_coordenada(id_libro, longuitud , latitud):
        bibliotecas = []
        db_connection = DatabaseConnection()
        db_connection.connect()
        
        query = """
        SELECT
        b.id,
        b.nombre,
        b.longitud,
        b.latitud,
        ST_Distance(
            ST_SetSRID(ST_MakePoint(b.longitud::float, b.latitud::float), 4326)::geography, 
            ST_SetSRID(ST_MakePoint(%s, %s), 4326)::geography
        ) AS distancia
         FROM bibliotecas_inf b
         JOIN biblioteca_libro bl ON b.id = bl.biblioteca_id
         WHERE bl.libro_id = %s
         ORDER BY
         distancia 
        LIMIT 5;"""
        params = (longuitud, latitud, id_libro) 
        cursor = db_connection.execute(query, params )
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        bibliotecas = [dict(zip(columns, row)) for row in rows]
        cursor.close()
        db_connection.disconnect()
        return bibliotecas

In [11]:
get_bibliotecas_by_id_libro_coordenada('Minp0000103871',-74.0652501,4.8563907)

connect to database


[{'id': 18,
  'nombre': 'Biblioteca Pública Virgilio Barco',
  'longitud': '-74.08845317',
  'latitud': '4.6570833700000005',
  'distancia': 22189.59729093},
 {'id': 30,
  'nombre': 'Biblioteca Universitaria de la Nacional',
  'longitud': '-74.08317329',
  'latitud': '4.6353275',
  'distancia': 24526.31894494},
 {'id': 9,
  'nombre': 'Biblioteca Nacional de Colombia',
  'longitud': '-74.06856233',
  'latitud': '4.6095469300000005',
  'distancia': 27298.91095248},
 {'id': 41,
  'nombre': 'Biblioteca Nacional de Colombia',
  'longitud': '-74.06856233',
  'latitud': '4.6095469300000005',
  'distancia': 27298.91095248},
 {'id': 12,
  'nombre': 'Biblioteca Universitaria de los Andes',
  'longitud': '-74.06553638',
  'latitud': '4.60204063',
  'distancia': 28126.51299764}]